In [1]:
import json
import matplotlib.pyplot as plt
import os

from skimage.io import imread

from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

from efficientnet.keras import EfficientNetB0
from efficientnet.keras import preprocess_input

from utils import preprocess_data
from efficientnet_classifier import make_image_generator, create_aug_generator

RANDOM_SEED = 10000
MODEL = 'efficientnet_classifier'


Using TensorFlow backend.


In [3]:
with open('config.json', 'r') as file:
    configs = json.load(file)
    
model_configs = configs['CLASSIFIER']

balanced = model_configs['BALANCE_DATA']
batch_size = model_configs['BATCH_SIZE']
steps_per_epoch = model_configs['STEPS_PER_EPOCH']
img_size = model_configs['IMG_SIZE']
path_checkpoint = model_configs['PATH_CHECKPOINT']


data_dir = configs["DATA_DIRECTORY"]
train_image_dir = os.path.join(data_dir, 'train_v2')

corrupted_images = configs['EXCLUDE_IMAGES']
train_df, test_df = preprocess_data(
    os.path.join(data_dir, 'train_ship_segmentations_v2.csv.zip'),
    corrupted_images,
    n_train=0.85,
    balanced=balanced
)

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

K.set_image_data_format('channels_last')

model = load_model(f'{path_checkpoint}/{MODEL}.hdf5')


In [4]:
train_eval_generator = create_aug_generator(
    make_image_generator(train_df, _batch_size=batch_size, _img_size=img_size, _train_image_dir=train_image_dir), 
    data_generator
)


In [5]:
model.evaluate_generator(train_eval_generator, steps=steps_per_epoch)


[0.31314143538475037, 0.9286249876022339]

In [6]:
test_eval_generator = create_aug_generator(
    make_image_generator(test_df, _batch_size=batch_size, _img_size=img_size, _train_image_dir=train_image_dir), 
    data_generator
)


In [7]:
model.evaluate_generator(test_eval_generator, steps=steps_per_epoch)


[0.006274908781051636, 0.9441249966621399]